<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据集的下载及介绍" data-toc-modified-id="数据集的下载及介绍-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>数据集的下载及介绍</a></span></li><li><span><a href="#调用库" data-toc-modified-id="调用库-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>调用库</a></span></li><li><span><a href="#读取店铺" data-toc-modified-id="读取店铺-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>读取店铺</a></span><ul class="toc-item"><li><span><a href="#抽取商店的id" data-toc-modified-id="抽取商店的id-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>抽取商店的id</a></span></li></ul></li><li><span><a href="#读取评论" data-toc-modified-id="读取评论-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>读取评论</a></span></li><li><span><a href="#提取正负样本" data-toc-modified-id="提取正负样本-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>提取正负样本</a></span><ul class="toc-item"><li><span><a href="#保存数据" data-toc-modified-id="保存数据-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>保存数据</a></span></li></ul></li></ul></div>

情感分析是文本分类的一个分支，是对带有情感色彩（褒义贬义/正向负向）的主观性文本进行分析，以确定该文本或者用户的观点、喜好、情感倾向。    
医疗领域同样存在情感分析的应用场景。

# 数据集的下载及介绍

> **数据集下载**
>> **本次实验**使用的是从***`kaggle`***上下载的数据    
- [kaggle_yelp数据集下载](https://www.kaggle.com/yelp-dataset/yelp-dataset)    
也可以到官网下载，数据有一定的不同    
- [yelp官网](https://www.yelp.com/dataset)

> 样例说明
>> 非常建议看一下样例说明    
- [yelp样例说明](https://www.yelp.com/dataset/documentation/main)

# 调用库

In [1]:
import json
import os
from tqdm import tqdm
import gc
import re
import pandas as pd

# 读取店铺

In [2]:
s = "../../dataset/archive/yelp_academic_dataset_business.json"

data_business = []

with open(s, 'r', encoding='UTF-8') as load_f:
    for line in tqdm(load_f):
        data_business.append(json.loads(line))

160585it [00:03, 52131.02it/s]


In [3]:
# 商店信息的数量
len(data_business)

160585

In [4]:
# 查看商店信息的存储格式
data_business[0]

{'business_id': '6iYb2HFDywm3zjuRg0shjw',
 'name': 'Oskar Blues Taproom',
 'address': '921 Pearl St',
 'city': 'Boulder',
 'state': 'CO',
 'postal_code': '80302',
 'latitude': 40.0175444,
 'longitude': -105.2833481,
 'stars': 4.0,
 'review_count': 86,
 'is_open': 1,
 'attributes': {'RestaurantsTableService': 'True',
  'WiFi': "u'free'",
  'BikeParking': 'True',
  'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
  'BusinessAcceptsCreditCards': 'True',
  'RestaurantsReservations': 'False',
  'WheelchairAccessible': 'True',
  'Caters': 'True',
  'OutdoorSeating': 'True',
  'RestaurantsGoodForGroups': 'True',
  'HappyHour': 'True',
  'BusinessAcceptsBitcoin': 'False',
  'RestaurantsPriceRange2': '2',
  'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}",
  'HasTV': 'True',
  'Alcohol': "'beer_and_wine'",
  'GoodFo

In [5]:
# 找到 categories 中包括 str1 & str2 这两个类的商店
str1 = 'Health & Medical'
str2 = 'Doctors'

business_hmd = []
for i in tqdm(range(len(data_business))):
    if data_business[i]["categories"] != None: # categories 不为空值
        flag = data_business[i]["categories"].rfind(str1) # 找不到 str1 就返回 -1
        if flag != -1:
            flag = data_business[i]["categories"].rfind(str2) # 找不到 str2 就返回 -1
            if flag != -1:
                business_hmd.append(data_business[i])

100%|███████████████████████████████████████████████████████████████████| 160585/160585 [00:00<00:00, 894524.83it/s]


In [6]:
len(business_hmd)

4700

In [7]:
del(data_business) # 清内存，删除列表
gc.collect()

49

In [8]:
business_hmd[0]

{'business_id': 's8eTWEtW5WcnL2kUxrOVmw',
 'name': 'Austin Regional Clinic: ARC Quarry Lake',
 'address': '4515 Seton Center Pkwy, Ste 220',
 'city': 'Austin',
 'state': 'TX',
 'postal_code': '78759',
 'latitude': 30.4016572,
 'longitude': -97.7429434,
 'stars': 3.5,
 'review_count': 48,
 'is_open': 1,
 'attributes': {'ByAppointmentOnly': 'True',
  'AcceptsInsurance': 'True',
  'BusinessAcceptsCreditCards': 'True'},
 'categories': 'Diagnostic Services, Doctors, Internal Medicine, Pediatricians, Laboratory Testing, Health & Medical, Family Practice, Surgeons',
 'hours': {'Monday': '8:0-17:0',
  'Tuesday': '8:0-17:0',
  'Wednesday': '8:0-17:0',
  'Thursday': '8:0-17:0',
  'Friday': '8:0-17:0'}}

## 抽取商店的id

In [9]:
business_hmd_id = []
for element in tqdm(business_hmd):
    business_hmd_id.append(element["business_id"])
len(business_hmd_id)

100%|██████████████████████████████████████████████████████████████████████| 4700/4700 [00:00<00:00, 1570650.05it/s]


4700

In [10]:
del(business_hmd) # 清内存
gc.collect()

69

# 读取评论

In [11]:
s = "../../dataset/archive/yelp_academic_dataset_review.json"

data_review = []

with open(s, 'r', encoding='UTF-8') as load_f:
    for line in tqdm(load_f):
        data_review.append(json.loads(line))

8635403it [01:39, 86883.27it/s] 


In [12]:
len(data_review)

8635403

In [13]:
data_review[0]

{'review_id': 'lWC-xP3rd6obsecCYsGZRg',
 'user_id': 'ak0TdVmGKo4pwqdJSTLwWw',
 'business_id': 'buF9druCkbuXLX526sGELQ',
 'stars': 4.0,
 'useful': 3,
 'funny': 1,
 'cool': 1,
 'text': "Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a 

In [14]:
reviews_hmd = []
for i in tqdm(range(len(data_review))):
    if data_review[i]['business_id'] in business_hmd_id:
        reviews_hmd.append(data_review[i])

len(reviews_hmd)

100%|███████████████████████████████████████████████████████████████████| 8635403/8635403 [28:05<00:00, 5122.99it/s]


95108

```python
del(data_review) # 清内存
gc.collect()
```

# 提取正负样本

In [21]:
pos = []
neg = []
mid = []

`cleanReview` 正则匹配 `\n` & `\`，并删除掉

In [22]:
def cleanReview(subject):
    newSubject = re.sub(r'\n', '', subject)
    newSubject = re.sub(r"\'", "'", newSubject)
    return newSubject

In [23]:
for element in tqdm(reviews_hmd):
    if element['stars'] >= 4.0:
        pos.append(cleanReview(element['text']))
    elif element['stars'] <=2.0:
        neg.append(cleanReview(element['text']))
    else:
        mid.append(cleanReview(element['text']))


 89%|██████████████████████████████████████████████████████████████▍       | 84804/95108 [00:40<00:00, 13177.95it/s]

In [24]:
print('pos: ', len(pos), '\n', 'neg: ', len(neg), '\n', 'mid: ', len(mid))

pos:  58782 
 neg:  33059 
 mid:  3267


In [25]:
len(pos) + len(neg) # 查看这两个分类一共有多少条

91841

In [26]:
df_pos = pd.DataFrame(columns=['']) # 创建dataframe， 定义一个没有名字的空列
df_neg = pd.DataFrame(columns=[''])
df_mid = pd.DataFrame(columns=[''])

df_pos[''] = pos # 给空列赋值
df_neg[''] = neg
df_mid[''] = mid

In [27]:
df_pos[:5]

,
0,I just had my lasik done last week and I am re...
1,I highly recommend TLC Laser for anyone lookin...
2,This was one of the few physical therapy place...
3,This is my second time getting a breast augmen...
4,I've been going off again and on again to Pro ...


In [28]:
df_neg[:5]

,
0,I've been to this location twice and couldn't ...
1,Used to be great. But has gone downhill since ...
2,An open letter to TCV:Recently I missed an app...
3,I was diagnosed with epilepsy over 10 years ag...
4,"Is this business, still in business? I've call..."


In [29]:
df_mid[:5]

,
0,On 11/28/17 I saw Dr. Gay in Pulmonology who r...
1,I love the convenience of ZoomCare. I've had s...
2,I went here for a prenatal massage and unfortu...
3,My young son and I both receive our medical ca...
4,I've been twice and I had two very different e...


## 保存数据

\n 是有意义的，换行 \\n才能到正确路径，记得先在上级目录创建`大数据概论`文件夹

In [34]:
# 保存数据
df_pos.to_excel('.\大数据概论\pos.xls', index=False, header=0) # 不保存索引和列名
df_neg.to_excel('.\大数据概论\\neg.xls', index=False, header=0)
df_mid.to_excel('.\大数据概论\\mid.xls', index=False, header=0)

我们将`正类`和`负类`取到一样的值，这样有助于数据的平衡     

> 打个比方，机械大多数都是男生     
>> 如果要预测机械学生的性别，模型只要将所有输入都预测为男生准确率就已经非常高了   
>>> 数据平衡还是非常重要的

而且我们的正类和负类数量相同，这样会使得**acc**表达的意义是很准确的，故，最后评价模型的时候**不再使用精准率、召回率和F1分数这些评价指标**，主要是***懒***

In [35]:
df_pos_30000 = df_pos[:30000]
df_neg_30000 = df_neg[:30000]
df_pos_30000.to_excel('.\大数据概论\pos_30000.xls', index=False, header=0)
df_neg_30000.to_excel('.\大数据概论\\neg_30000.xls', index=False, header=0)